## EJERCICIO 3


**Enunciado**: Usa la clase PairWiseAligner de Biopython para alinear secuencias
de aminoácidos (proteínas) y obtener la puntuación y el alineamiento óptimo
en estos dos escenarios.




a) Generar las secuencias aleatoriamente (de forma automática no manual).


b) Obtener las secuencias a partir de ficheros obtenidos de bases de
datos biológicas (como Uniprot). Explicar cómo se obtuvieron.
Prueba a usar diferentes matrices de sustitución, como BLOSUM62, PAM250 o
una matriz personalizada. Compara los resultados obtenidos con diferentes
matrices e investiga las ventajas y desventajas de cada una.

***Los siguientes métodos son comunes a ambos apartados.***

In [16]:
import random
from pathlib import Path
from typing import Optional, Tuple, List

from Bio import SeqIO
from Bio.Align import PairwiseAligner
from Bio.Align import substitution_matrices as sm

AA20 = "ACDEFGHIKLMNPQRSTVWY"
random.seed(42)


def generar_proteina_aleatoria(longitud: int) -> str:
    return "".join(random.choice(AA20) for _ in range(longitud))


def limpiar_proteina(seq: str, reemplazo: str = "X") -> str:
    seq = seq.upper()
    return "".join(c if c in AA20 else reemplazo for c in seq)


def calcular_identidad_y_huecos(secuencia1_alineada: str, secuencia2_alineada: str) -> Tuple[float, int, int]:
    coincidencias = sum(
        1 for a, b in zip(secuencia1_alineada, secuencia2_alineada)
        if a == b and a != "-" and b != "-"
    )
    posiciones_alineadas = sum(
        1 for a, b in zip(secuencia1_alineada, secuencia2_alineada)
        if a != "-" and b != "-"
    )
    identidad = (coincidencias / posiciones_alineadas) * 100 if posiciones_alineadas else 0.0
    huecos = secuencia1_alineada.count("-") + secuencia2_alineada.count("-")
    return round(identidad, 2), coincidencias, huecos


def reconstruir_con_huecos(alignment, seq_a: str, seq_b: str) -> Tuple[str, str]:
    A, B = [], []
    coords = alignment.coordinates
    for j in range(coords.shape[1] - 1):
        a0, a1 = coords[0, j], coords[0, j + 1]
        b0, b1 = coords[1, j], coords[1, j + 1]
        while a0 < a1 and b0 < b1:
            A.append(seq_a[a0]); B.append(seq_b[b0])
            a0 += 1; b0 += 1
        while a0 < a1:
            A.append(seq_a[a0]); B.append('-'); a0 += 1
        while b0 < b1:
            A.append('-'); B.append(seq_b[b0]); b0 += 1
    return "".join(A), "".join(B)


def formatear_bloques(alA: str, alB: str, etiqueta_arriba="Proteína A", etiqueta_abajo="Proteína B",
                      ancho=60, off_a=0, off_b=0) -> str:
    i = 0
    out: List[str] = []
    while i < len(alA):
        sA = alA[i:i + ancho]
        sB = alB[i:i + ancho]
        mid = "".join(
            '|' if (a == b and a != '-' and b != '-') else ('.' if a != '-' and b != '-' else ' ')
            for a, b in zip(sA, sB)
        )
        startA = off_a + sum(1 for c in alA[:i] if c != '-') + 1
        endA   = startA + sum(1 for c in sA      if c != '-') - 1
        startB = off_b + sum(1 for c in alB[:i] if c != '-') + 1
        endB   = startB + sum(1 for c in sB      if c != '-') - 1

        out.append(f"{etiqueta_arriba:<12} {startA:>6} {sA} {endA}")
        out.append(f"{'':<12} {'':>6} {mid}")
        out.append(f"{etiqueta_abajo:<12} {startB:>6} {sB} {endB}\n")
        i += ancho
    return "\n".join(out).rstrip()


def cargar_matriz(nombre_o_path: Optional[str]):
    if nombre_o_path is None:
        return sm.load("BLOSUM62")

    try:
        return sm.load(nombre_o_path)
    except Exception:
        pass

    path = Path(nombre_o_path)
    if not path.exists():
        raise FileNotFoundError(f"No se encuentra la matriz: {nombre_o_path}")

    with path.open() as fh:
        rows = [line.strip() for line in fh if line.strip() and not line.strip().startswith("#")]
    header = rows[0].split()
    alphabet = "".join(header)
    n = len(header)
    arr = [[0] * n for _ in range(n)]
    for i, line in enumerate(rows[1:1 + n]):
        parts = line.split()
        if parts[0] != header[i]:
            raise ValueError("Fila/columna no alineadas en la matriz personalizada.")
        scores = list(map(float, parts[1:1 + n]))
        for j, val in enumerate(scores):
            arr[i][j] = val

    mat = sm.Array(alphabet=alphabet, dims=2)
    for i, a in enumerate(alphabet):
        for j, b in enumerate(alphabet):
            mat[(a, b)] = arr[i][j]
    return mat


def ejecutar_alineamiento(
    secuencia_a: str,
    secuencia_b: str,
    descripcion: str,
    modo: str,
    matriz: str,
    gap_lineal: Optional[float],
    open_gap: float,
    ext_gap: float,
    etiqueta_arriba="Proteína A",
    etiqueta_abajo="Proteína B",
):
    alineador = PairwiseAligner()
    alineador.mode = modo
    subm = cargar_matriz(matriz)
    alineador.substitution_matrix = subm
    if gap_lineal is not None:
        alineador.open_gap_score = gap_lineal
        alineador.extend_gap_score = gap_lineal
    else:
        alineador.open_gap_score = open_gap
        alineador.extend_gap_score = ext_gap

    mejor = alineador.align(secuencia_a, secuencia_b)[0]
    alA, alB = reconstruir_con_huecos(mejor, secuencia_a, secuencia_b)
    identidad, nmatch, ngaps = calcular_identidad_y_huecos(alA, alB)

    a_ini = mejor.aligned[0][0][0] if mejor.aligned[0].size else 0
    b_ini = mejor.aligned[1][0][0] if mejor.aligned[1].size else 0

    print("\n" + "═" * 78)
    print(f"Configuración: {descripcion}")
    print(f"Modo: {modo.upper()} | Matriz: {matriz}")
    print("─" * 78)
    print(
        formatear_bloques(
            alA, alB,
            etiqueta_arriba=etiqueta_arriba,
            etiqueta_abajo=etiqueta_abajo,
            ancho=60, off_a=a_ini, off_b=b_ini
        )
    )
    print("─" * 78)
    print(f"Puntuación total         : {mejor.score:.3f}")
    print(f"Porcentaje de identidad  : {identidad}%")
    print(f"Número de coincidencias  : {nmatch}")
    print(f"Número total de huecos   : {ngaps}")
    print("═" * 78)


def comparar_matrices(secuencia_a: str, secuencia_b: str, titulo: str,
                      matrices: List[str], modo: str):
    print("\n" + "=" * 78)
    print(f"{titulo.center(78)}")
    print("=" * 78)
    configuraciones = []
    for M in matrices:
        configuraciones.append((f"{modo.capitalize()} (afín)   {M}  open=-10  ext=-0.5", modo, M, None, -10, -0.5))
        configuraciones.append((f"{modo.capitalize()} (lineal) {M}  gap=-2",             modo, M, -2,  None, None))

    for desc, md, M, gap_lineal, og, eg in configuraciones:
        ejecutar_alineamiento(
            secuencia_a, secuencia_b, desc, md,
            matriz=M,
            gap_lineal=gap_lineal,
            open_gap=og if og is not None else -10,
            ext_gap=eg if eg is not None else -0.5,
            etiqueta_arriba="Proteína A",
            etiqueta_abajo="Proteína B",
        )


def leer_secuencia_fasta(ruta_fichero: Path) -> str:
    registro = next(SeqIO.parse(str(ruta_fichero), "fasta"))
    seq = str(registro.seq)
    return limpiar_proteina(seq)


## 1. Generación de secuencias

Para analizar el comportamiento del alineamiento global, se generaron dos proteínas aleatorias:

- **Proteína A**: longitud **60**
- **Proteína B**: longitud **48**

Estas secuencias no tienen relación evolutiva real y sirven como caso de prueba para estudiar cómo responden distintos parámetros de alineamiento.

---

## 2. Objetivo del análisis

El propósito es comparar ambas proteínas bajo:

- Distintas **matrices de sustitución**, y  
- Diferentes **modelos de penalización de huecos**

con el fin de observar cómo cada combinación afecta:

- La puntuación del alineamiento  
- El número de huecos  
- El porcentaje de identidad  
- La estructura final del alineamiento

---

## 3. Matrices de sustitución empleadas

Las matrices evaluadas fueron:

- **BLOSUM62**
- **PAM250**

---

## 4. Modo de alineamiento

El alineamiento se realizó en modo:

- **global (Needleman–Wunsch)**  
  → Obliga a alinear las secuencias completas de inicio a fin.

---

## 5. Modelos de penalización de huecos

Cada matriz (BLOSUM62 y PAM250) se evaluó con dos modelos de penalización:

---

### 5.1 Modelo afín de huecos

Características:

- **open = –10**
- **ext = –0.5**
- Representa mejor procesos evolutivos reales  
- Penaliza fuerte abrir un hueco, pero extenderlo cuesta menos  
- Permite huecos largos con coste reducido

Configuraciones correspondientes:

- Global (afín)   BLOSUM62  open=-10  ext=-0.5
- Global (afín)   PAM250    open=-10  ext=-0.5

---

### 5.2 Modelo lineal de huecos

Características:

- **gap = –2**
- Penaliza cada posición del hueco de forma uniforme  
- Más simple, menos biológicamente realista  
- Tiende a generar alineamientos más rígidos

Configuraciones correspondientes:
- Global (lineal) BLOSUM62  gap=-2
- Global (lineal) PAM250    gap=-2


In [17]:
lenA = 60 
lenB = 48   
matrices = ["BLOSUM62", "PAM250"] 
modo = "global" 

prot_a = generar_proteina_aleatoria(lenA)
prot_b = generar_proteina_aleatoria(lenB)

print("\nProteínas aleatorias generadas:")
print(f" - Proteína A (longitud {len(prot_a)}): {prot_a}")
print(f" - Proteína B (longitud {len(prot_b)}): {prot_b}")

comparar_matrices(
    prot_a,
    prot_b,
    titulo="(a) Proteínas aleatorias",
    matrices=matrices,
    modo=modo,
)


Proteínas aleatorias generadas:
 - Proteína A (longitud 60): EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRVEPDVLYNWHDCILD
 - Proteína B (longitud 48): IEPKRNGNNHKDYGVIGRPKVIMCICMPKDHWMHSPRFKFIVVKWQWP

                           (a) Proteínas aleatorias                           

══════════════════════════════════════════════════════════════════════════════
Configuración: Global (afín)   BLOSUM62  open=-10  ext=-0.5
Modo: GLOBAL | Matriz: BLOSUM62
──────────────────────────────────────────────────────────────────────────────
Proteína A        6 EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKF---------H-MEDPENNYKCRVEPDV 55
                         .|.......|..|.|       .|..|.         | |..|  ..|..|   |
Proteína B        1 -----IEPKRNGNNHKDYGV-------IGRPKVIMCICMPKDHWMHSP--RFKFIV---V 43

Proteína A       56 LYNWHDCILD 65
                    ...|     .
Proteína B       44 KWQW-----P 48
──────────────────────────────────────────────────────────────────────────────
Puntuación total         : -23.500
Porce

| Nº | Conclusión                   | Explicación en palabras simples                                                                   |
|----|------------------------------|----------------------------------------------------------------------------------------------------|
| **1** | Los modelos lineales dan mejores puntuaciones | Penalizan menos los huecos, por lo que el alineamiento parece más favorable aunque no haya relación real. |
| **2** | PAM250 puntúa más alto que BLOSUM62 | PAM250 acepta más sustituciones y está diseñada para proteínas muy diferentes, por eso da puntuaciones más altas. |
| **3** | La identidad es baja (21–38 %) | Es normal: las proteínas fueron generadas aleatoriamente y no comparten un origen evolutivo. |
| **4** | Hay muchos huecos (30–34) | El algoritmo introduce muchos huecos para intentar forzar coincidencias, indicando falta de homología real. |
| **5** | La puntuación más alta es Global + lineal + PAM250 | No significa mejor alineamiento biológico: esta configuración es simplemente más permisiva. |

In [18]:
ruta_fasta1 = Path("fasta/mus_musculus_myocardin.fasta")
ruta_fasta2 = Path("fasta/chlorocebus_sabaeus_myocardin.fasta")

if ruta_fasta1.exists() and ruta_fasta2.exists():
    prot_a_db = leer_secuencia_fasta(ruta_fasta1)
    prot_b_db = leer_secuencia_fasta(ruta_fasta2)

    print("\nArchivos FASTA cargados correctamente:")
    print(f" - {ruta_fasta1.name} (longitud: {len(prot_a_db)})")
    print(f" - {ruta_fasta2.name} (longitud: {len(prot_b_db)})")

    comparar_matrices(
        prot_a_db,
        prot_b_db,
        titulo="(b) Secuencias de bases de datos (p. ej., UniProt)",
        matrices=matrices,
        modo=modo,
    )
else:
    print("️    No se han encontrado los ficheros FASTA indicados.")
    print("   Asegúrate de que existen, o cambia los nombres en 'ruta_fasta1' y 'ruta_fasta2'.")


Archivos FASTA cargados correctamente:
 - mus_musculus_myocardin.fasta (longitud: 1080)
 - chlorocebus_sabaeus_myocardin.fasta (longitud: 986)

              (b) Secuencias de bases de datos (p. ej., UniProt)              

══════════════════════════════════════════════════════════════════════════════
Configuración: Global (afín)   BLOSUM62  open=-10  ext=-0.5
Modo: GLOBAL | Matriz: BLOSUM62
──────────────────────────────────────────────────────────────────────────────
Proteína A        1 MIDSSKKQPQGFPEILTAEDFEPFKEKECLEGSNQKSLKEVLQLRLQQRRTREQLVDQGI 60
                    |            ..|..|.......|          ...|||||||||||.|||..||.
Proteína B        1 M------------TLLGSEHSLLIRSK----------FRSVLQLRLQQRRTQEQLANQGL 38

Proteína A       61 MPPLKSPAAFHEQIKSLERARTENFLKHKIRSRPDRSELVRMHILEETFAEPSLQATQMK 120
                    .||||.||.||||.|.||.....|.||.|.|.|.....||.||||....||.|....|||
Proteína B       39 IPPLKRPAEFHEQRKHLESDKAKNSLKRKARNRCNSANLVNMHILQASTAERSIPTAQMK 98

Proteína A      121 LKRA

### Hemos tratado los FASTA:

A continuación se muestran las proteínas utilizadas en el apartado (b), obtenidas de UniProt y posteriormente procesadas en formato FASTA.


<p align="center">
  <img src="images/chlorocebus_sabaeus.png" width="320">
</p>

#### *Chlorocebus sabaeus* — Myocardin  
(Conocido como **Green monkey**)


<p align="center">
  <img src="images/mus_musculus.jpg" width="320">
</p>

#### *Mus Musculus* — Myocardin  
(Conocido como **Laboratory Mouse**)


### ¿Cómo se obtuvieron las secuencias?

Las secuencias proteicas se obtuvieron siguiendo estos pasos:

- Se buscaron en **UniProt** las proteínas de interés.
- En cada entrada de proteína, se utilizó el botón **“Download” → FASTA”** para descargar la secuencia aminoacídica.
- Los ficheros FASTA descargados se guardaron en el mismo directorio que este notebook, dentro de la carpeta **`fasta/`**:

  - `chlorocebus_sabaeus_myocardin.fasta`: https://rest.uniprot.org/uniprotkb/A0A0D9R9R8.fasta
  - `mus_musculus_myocardin.fasta`: https://rest.uniprot.org/uniprotkb/P59759.fasta
